**Note:** Make sure you go through the [Setup Notebook](penguins-setup.ipynb) notebook once at the start of the program.

In [2]:
# !pip install -q --upgrade pip
# !pip install -q --upgrade awscli boto3
!pip install -q --upgrade scikit-learn==0.23.2
# !pip install -q --upgrade PyYAML==6.0
!pip install -q --upgrade pip
!pip install -q --upgrade awscli boto3
!pip install -q --upgrade sagemaker==2.173.0
!pip show sagemaker

Name: sagemaker
Version: 2.173.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.8/site-packages
Requires: attrs, boto3, cloudpickle, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, PyYAML, schema, smdebug-rulesconfig, tblib
Required-by: 


In [3]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

In [4]:
import os
import numpy as np
import json
import numpy as np
import tempfile
import time
import tarfile

from constants import *
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.parameters import ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter, ContinuousParameter
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession

from sagemaker import ModelPackage
from sagemaker.pytorch import PyTorchProcessor
from sagemaker.model import Model
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo, ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.model_step import ModelStep

MODEL_PACKAGE_GROUP = "penguins"
ENDPOINT="penguins-endpoint"

In [5]:
%%writefile {CODE_FOLDER}/preprocessor.py

## Preprocessing script
import os
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump


# This is the location where the SageMaker Processing job
# will save the input dataset.
BASE_DIRECTORY = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIRECTORY) / "input" / "data.csv"


def _save_splits(base_directory, train, validation, test):
    """
    One of the goals of this script is to output the three
    dataset splits. This function will save each of these
    splits to disk.
    """

    train_path = Path(base_directory) / "train"
    validation_path = Path(base_directory) / "validation"
    test_path = Path(base_directory) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        validation_path / "validation.csv", header=False, index=False
    )
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def _save_pipeline(base_directory, pipeline):
    """
    Saves the Scikit-Learn pipeline that we used to
    preprocess the data.
    """
    pipeline_path = Path(base_directory) / "pipeline"
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / "pipeline.pkl", "wb"))


def _save_classes(base_directory, classes):
    """
    Saves the list of classes from the dataset.
    """
    path = Path(base_directory) / "classes"
    path.mkdir(parents=True, exist_ok=True)

    np.asarray(classes).tofile(path / "classes.csv", sep=",")


def _save_baseline(base_directory, df_train, df_test):
    """
    During the data and quality monitoring steps, we will need a baseline
    to compute constraints and statistics. This function will save that
    baseline to the disk.
    """

    for split, data in [("train", df_train), ("test", df_test)]:
        baseline_path = Path(base_directory) / f"{split}-baseline"
        baseline_path.mkdir(parents=True, exist_ok=True)

        df = data.copy().dropna()
        df.to_json(
            baseline_path / f"{split}-baseline.json", orient="records", lines=True
        )


def preprocess(base_directory, data_filepath):
    """
    Preprocesses the supplied raw dataset and splits it into a train,
    validation, and a test set.
    """

    df = pd.read_csv(data_filepath)

    numeric_features = df.select_dtypes(include=['float64']).columns.tolist()
    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("encoder", OneHotEncoder()),
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("numeric", numeric_transformer, numeric_features),
            ("categorical", categorical_transformer, ["island"]),
        ]
    )

    pipeline = Pipeline(
        steps=[
            ("preprocessing", preprocessor)
        ]
    )

    df.drop(["sex"], axis=1, inplace=True)
    df = df.sample(frac=1, random_state=42)

    df_train, temp = train_test_split(df, test_size=0.3)
    df_validation, df_test = train_test_split(temp, test_size=0.5)

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(df_train.species)
    y_validation = label_encoder.transform(df_validation.species)
    y_test = label_encoder.transform(df_test.species)
    
    _save_baseline(base_directory, df_train, df_test)

    df_train = df_train.drop(["species"], axis=1)
    df_validation = df_validation.drop(["species"], axis=1)
    df_test = df_test.drop(["species"], axis=1)

    X_train = pipeline.fit_transform(df_train)
    X_validation = pipeline.transform(df_validation)
    X_test = pipeline.transform(df_test)

    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)

    _save_splits(base_directory, train, validation, test)
    _save_pipeline(base_directory, pipeline=pipeline)
    _save_classes(base_directory, label_encoder.classes_)


if __name__ == "__main__":
    preprocess(BASE_DIRECTORY, DATA_FILEPATH)


Overwriting code/preprocessor.py


In [6]:
# Test preprocessing script

from preprocessor import preprocess

with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    print(f"Folders: {os.listdir(directory)}")

Folders: ['train-baseline', 'test-baseline', 'train', 'validation', 'test', 'pipeline', 'classes']


In [7]:
# preprocess_data_step parameters
dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data.csv",
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f"{S3_LOCATION}/preprocessing",
)

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

In [8]:
# Define preprocessing step
preprocess_data_step = ProcessingStep(
    name="preprocess-data",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
        ProcessingOutput(output_name="classes", source="/opt/ml/processing/classes", destination=preprocessor_destination),
        ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline"),
        ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline"),
    ],
    code=f"{CODE_FOLDER}/preprocessor.py",
    cache_config=cache_config
)



In [9]:
%%writefile {CODE_FOLDER}/train.py
#  Pytorch training script
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

import os
import argparse

from pathlib import Path
import numpy as np
import pandas as pd

class PenguinModel(nn.Module):
    def __init__(self, input_shape):
        super(PenguinModel, self).__init__()
        self.fc1 = nn.Linear(in_features=input_shape, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=8)
        self.fc3 = nn.Linear(in_features=8, out_features=3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # Apply softmax along dimension 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    
    # Prediction function
    def predict(self, input_data):
        input_data_torch = torch.tensor(input_data.values, dtype=torch.float32)
        with torch.no_grad():
            self.eval()  # Set the model to evaluation mode
            output = self(input_data_torch)
        return output

def train(base_directory, train_path, validation_path, epochs=50, batch_size=32, learning_rate=0.01):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]] # Get the last column of the training dataset
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
   
    # Convert data to PyTorch tensors
    X_train_torch = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
    X_validation_torch = torch.tensor(X_validation.values, dtype=torch.float32)
    y_validation_torch = torch.tensor(y_validation.values, dtype=torch.long)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_torch, y_train_torch)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    # Initialize the model, loss, and optimizer
    model = PenguinModel(X_train.shape[1])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        for inputs, targets in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            total_loss += loss 
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == targets).sum().item()
            total_samples += targets.size(0)
         # Calculate accuracy and average loss for the epoch
        epoch_loss = total_loss / len(train_loader)
        epoch_accuracy = correct_predictions / total_samples

        print(f"Epoch [{epoch+1}/{epochs}] - loss: {epoch_loss:.4f}, val_accuracy: {epoch_accuracy:.4f}")
        
    # Save model
    model_path = Path(base_directory) / 'model' / '001'
    model_path.mkdir(parents=True,exist_ok=True)
    
    torch.save(model.state_dict(), model_path / 'model.pth')
    
    print(f'Model saved: {model_path.resolve()}/model.pth')
   
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list: 
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    # SageMaker will automatically create env variables(prefixed with SM_CHANNEL_) for the training inputs defined in the training step further below
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))  
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--learning_rate", type=float)
    parser.add_argument("--epochs", type=int)
    parser.add_argument("--batch_size", type=int)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        learning_rate=args.learning_rate,
        batch_size=args.batch_size
    )

Overwriting code/train.py


In [10]:
!cp {CODE_FOLDER}/train.py  {CODE_FOLDER}/endpoint/train.py

In [11]:
# Test preprocessing and training script combined.
from preprocessor import preprocess
# from train import train
from train import train

# Create a temporary directory to test the training.
with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=100,
        learning_rate=0.01
    )

[2023-09-14 17:05:05.068 pytorch-1-10-cpu-py38-ml-t3-medium-8265974f1f54da4fb1fd6ac71882:1199 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?


[2023-09-14 17:05:05.350 pytorch-1-10-cpu-py38-ml-t3-medium-8265974f1f54da4fb1fd6ac71882:1199 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Epoch [1/100] - loss: 1.0889, val_accuracy: 0.4059
Epoch [2/100] - loss: 1.0886, val_accuracy: 0.4059
Epoch [3/100] - loss: 1.0882, val_accuracy: 0.4059
Epoch [4/100] - loss: 1.0878, val_accuracy: 0.4059
Epoch [5/100] - loss: 1.0874, val_accuracy: 0.4059
Epoch [6/100] - loss: 1.0870, val_accuracy: 0.4059
Epoch [7/100] - loss: 1.0866, val_accuracy: 0.4059
Epoch [8/100] - loss: 1.0863, val_accuracy: 0.4059
Epoch [9/100] - loss: 1.0858, val_accuracy: 0.4059
Epoch [10/100] - loss: 1.0854, val_accuracy: 0.4059
Epoch [11/100] - loss: 1.0850, val_accuracy: 0.4059
Epoch [12/100] - loss: 1.0845, val_accuracy: 0.4059
Epoch [13/100] - loss: 1.0840, val_accuracy: 0.4059
Epoch [14/100] - loss: 1.0836, val_accuracy: 0.4059
Epoch [15/100] - loss: 1.0830, val_accuracy: 0.4059
Epoch [16/

In [128]:
# Define tuning step
objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 100),
    "batch_size": IntegerParameter(8, 16),
    "learning_rate": ContinuousParameter(0.007, 0.01),
}

estimator = PyTorch(
    entry_point=f"{CODE_FOLDER}/train.py",
    framework_version="1.8",
    instance_type="ml.m5.large",
    py_version="py36",
    instance_count=1,
    script_mode=True,
    
    # The default profiler rule includes a timestamp which will change each time
    # the pipeline is upserted, causing cache misses. Since we don't need
    # profiling, we can disable it to take advantage of caching.
    disable_profiler=True,

    role=role,
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

In [129]:
tune_model_step = TuningStep(
    name = "tune-model",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

In [130]:
%%writefile {CODE_FOLDER}/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd
import argparse


from pathlib import Path
# from tensorflow import keras
import torch

from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"
EVALUATION_NAME="evaluation"

# Had to repeat class here due to bug in using source_dir param in pytorch_processor.run
class PenguinModel(nn.Module):
    def __init__(self, input_shape):
        super(PenguinModel, self).__init__()
        self.fc1 = nn.Linear(in_features=input_shape, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=8)
        self.fc3 = nn.Linear(in_features=8, out_features=3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # Apply softmax along dimension 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

    # Prediction function
    def predict(self, input_data):
        input_data_torch = torch.tensor(input_data.values, dtype=torch.float32)
        with torch.no_grad():
            self.eval()  # Set the model to evaluation mode
            output = self(input_data_torch)        
        return output
    
def evaluate(model_path, test_path, output_path, evaluation_name):
    # The first step is to extract the model package so we can load 
    # it in memory.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    model = PenguinModel(X_test.shape[1])
    model.load_state_dict(torch.load(Path(model_path) / "001" / "model.pth"))

    predictions = np.argmax(model.predict(X_test), axis=-1)
    
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')
    num_samples = X_test.shape[0]
    
    # print(f"Accuracy: {accuracy}. Precision: {precision}, Recall: {recall}, F1: {f1}, num_samples: {num_samples}")

    # Let's create an evaluation report using the model accuracy.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
            "Precision": {
                "value": precision
            },
            "Recall": {
                "value": recall
            },
            "F1": {
                "value": f1
            },
            "num_samples": {
                "value": num_samples
            }
        },
    }
    print(evaluation_report)
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / f"{evaluation_name}.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--evaluation_name', type=str, dest='evaluation_name', default="evaluation")

    args, _ = parser.parse_known_args()
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH,
        evaluation_name=args.evaluation_name
    )

Overwriting code/evaluation.py


In [131]:
from preprocessor import preprocess
from train import train
from evaluation import evaluate

import json


with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
        evaluation_name="evaluation",
    )
    
    with open(Path(directory) / "evaluation" / f"evaluation.json", "r") as file:
        data = json.load(file)
        print(data)

Epoch [1/50] - loss: 1.0899, val_accuracy: 0.4100
Epoch [2/50] - loss: 1.0886, val_accuracy: 0.4100
Epoch [3/50] - loss: 1.0873, val_accuracy: 0.4100
Epoch [4/50] - loss: 1.0860, val_accuracy: 0.4100
Epoch [5/50] - loss: 1.0846, val_accuracy: 0.4100
Epoch [6/50] - loss: 1.0833, val_accuracy: 0.4100
Epoch [7/50] - loss: 1.0819, val_accuracy: 0.4100
Epoch [8/50] - loss: 1.0805, val_accuracy: 0.4100
Epoch [9/50] - loss: 1.0791, val_accuracy: 0.4100
Epoch [10/50] - loss: 1.0777, val_accuracy: 0.4100
Epoch [11/50] - loss: 1.0762, val_accuracy: 0.4100
Epoch [12/50] - loss: 1.0747, val_accuracy: 0.4100
Epoch [13/50] - loss: 1.0732, val_accuracy: 0.4100
Epoch [14/50] - loss: 1.0717, val_accuracy: 0.4100
Epoch [15/50] - loss: 1.0701, val_accuracy: 0.4100
Epoch [16/50] - loss: 1.0685, val_accuracy: 0.4100
Epoch [17/50] - loss: 1.0669, val_accuracy: 0.4100
Epoch [18/50] - loss: 1.0653, val_accuracy: 0.4100
Epoch [19/50] - loss: 1.0636, val_accuracy: 0.4100
Epoch [20/50] - loss: 1.0619, val_accura

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [132]:
pytorch_processor = PyTorchProcessor(
    base_job_name="penguins-evaluation-processor",
    framework_version="1.8",
    py_version="py36",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
    sagemaker_session=PipelineSession(),
)

# This is a workaround to a problem with the SageMaker SDK: 
# By default, the TensorFlowProcessor runs the script using
# /bin/bash as its entrypoint. We want to ensure we run it 
# using python3.
pytorch_processor.framework_entrypoint_command = ["python3"]

eval_winner_name = "evaluate-winner-model"

# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
def create_eval_report(report_name):
    return PropertyFile(
        name=report_name,
        output_name="evaluation",
        path=f"{report_name}.json",
    )

def create_eval_process_step(evaluation_name, report, top_k=0):
    return ProcessingStep(
        name=evaluation_name,
        processor=pytorch_processor,
        inputs=[
            ProcessingInput(source=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test"
            ),
            ProcessingInput(
                source=(
                    tune_model_step.get_top_model_s3_uri(top_k=top_k, s3_bucket=sagemaker_session.default_bucket()) 
                ),
                destination="/opt/ml/processing/model",
            )
        ],
        outputs=[
            ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=f"{S3_LOCATION}/evaluation"),
        ],
        code=f'{CODE_FOLDER}/evaluation.py',
        job_arguments=["--evaluation_name", evaluation_name],
        property_files=[report],
        cache_config=cache_config,
    )
    
eval_winner_report = create_eval_report(eval_winner_name)

eval_model_step = create_eval_process_step(eval_winner_name, 
                                        eval_winner_report)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [133]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="/", values=[
            eval_model_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'], f"{eval_winner_name}.json"]
        ),
        content_type="application/json",
    )
)

model_package_group_name = "penguins"

In [134]:
def get_model(top_k=0):
    return PyTorchModel(
        model_data=(
            tune_model_step.get_top_model_s3_uri(top_k, s3_bucket=PipelineSession().default_bucket())
        ),
        framework_version="1.8",
        py_version="py36",
        sagemaker_session=PipelineSession(),
        role=role,
    )

def model_registry_args(model,
                        model_metrics, 
                        approval_status="PendingManualApproval"):
    return model.register(
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=approval_status,
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )

model = get_model() 

args_approved = model_registry_args(model, 
                                    model_metrics, 
                                    "Approved")

args_pending = model_registry_args(model,
                                model_metrics)

register_step_approved = ModelStep(
    name="register-model-approved",
    step_args=args_approved,
)

register_step_pending = ModelStep(
    name="register-model-pending-approval",
    step_args=args_pending,
)


/opt/conda/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [135]:
accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.70
)

min_accuracy_threshold = ParameterFloat(
    name="accuracy_threshold_min", 
    default_value=0.50
)

def gte_eval_condition(step, report, accuracy):
    return ConditionGreaterThanOrEqualTo(
        left=JsonGet(
            step_name=step.name,
            property_file=report,
            json_path="metrics.accuracy.value"
        ),
        right=accuracy
    )

def lte_eval_condition(step, report, accuracy):
    return ConditionLessThanOrEqualTo(
        left=JsonGet(
            step_name=step.name,
            property_file=report,
            json_path="metrics.accuracy.value"
        ),
        right=accuracy
    )


gte_approved = gte_eval_condition(eval_model_step, 
                                eval_winner_report,
                                accuracy_threshold)

lte_min = lte_eval_condition(eval_model_step,
                            eval_winner_report,
                            min_accuracy_threshold)


gte_min = gte_eval_condition(eval_model_step,
                            eval_winner_report,
                            min_accuracy_threshold)

lte_approved = lte_eval_condition(eval_model_step,
                               eval_winner_report,
                               accuracy_threshold)

fail_step_min = FailStep(
    name="fail-min",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            min_accuracy_threshold
        ]
    ),
)

def create_condition_step(name, conditions, if_steps, else_steps=None):
    return ConditionStep(
        name=name,
        conditions=conditions,
        if_steps=if_steps,
        else_steps=else_steps
    )

# evaluate_tune_min = create_condition_step("check-min-accuracy", [condition_pending_approval_winner, condition_pending_approval_second],[fail_step_min])

check_min_step = create_condition_step("min-model-accuracy",
                                        [lte_min],
                                        [fail_step_min],)

approved_step = create_condition_step("approved-model-accuracy", 
                                        [gte_approved],
                                        [register_step_approved])
pending_step = create_condition_step("pending-model-accuracy",
                                        [lte_approved, gte_min],
                                        [register_step_pending],)

In [136]:
pipeline = Pipeline(
    name="penguins-best-model-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        accuracy_threshold,
        min_accuracy_threshold,
    ],
    steps=[
        preprocess_data_step, 
        tune_model_step,
        eval_model_step,
        check_min_step,
        approved_step,
        pending_step,
    ],
    pipeline_definition_config=pipeline_definition_config
)
# pipeline.delete()
pipeline.upsert(role_arn=role)
pipeline.start()


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:253909639528:pipeline/penguins-best-model-pipeline/execution/s23chs0r9o8i', sagemaker_session=<sagemaker.session.Session object at 0x7f4ec4698c10>)

In [137]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Downloader
from sagemaker.workflow.parameters import ParameterInteger
from sagemaker import ModelPackage


In [138]:
ENDPOINT_CODE_FOLDER = CODE_FOLDER / "endpoint"
Path(ENDPOINT_CODE_FOLDER).mkdir(parents=True, exist_ok=True)
sys.path.append
(f"./{ENDPOINT_CODE_FOLDER}")

'./code/endpoint'

We will include the inference code as part of the model assets to control the inference process on the SageMaker endpoint. SageMaker will automatically call the `handler()` function for every request to the endpoint.

In [139]:
%%writefile {ENDPOINT_CODE_FOLDER}/inference.py

from model import PenguinModel

import os
import json
import boto3
from pathlib import Path
import numpy as np
import pandas as pd

from pickle import load

s3 = boto3.resource("s3")

import torch
import torch.nn as nn
import torch.optim as optim

def _get_pipeline(directory=None):
    """
    This function returns the Scikit-Learn pipeline we used to transform the
    dataset.
    """
    if(directory is None):
        directory = os.environ("PREPROCESSING_DIR", "/tmp")
        
    pipeline_file = Path(directory) / "pipeline" / "pipeline.pkl"
    
    _download(pipeline_file)
    
    return load(open(pipeline_file, 'rb'))

def _get_class(prediction, directory):
    """
    This function returns the class name of a given prediction. 
    """
    classes_file = Path(directory) / "pipeline" / "classes.csv"
    _download(classes_file)
    
    with open(classes_file) as f:
        file = f.readlines()
        
    classes = list(map(lambda x: x.replace("'", ""), file[0].split(',')))
    return classes[prediction]

def _download(file):
    """
    This function will download a file from S3 if it doesn't already exist. The
    function will use the `S3_LOCATION` environment variable to determine the
    location of the file.
    """
    if(file.exists()):
        return
    
    s3_uri = os.environ.get("S3_LOCATION", f"s3://vmate-mlschool4/penguins/preprocessing")
        
    s3_parts = s3_uri.split('/', 3)
    bucket = s3_parts[2]
    key = s3_parts[3]
    
    s3.Bucket(bucket).download_file(f"{key}/{file.name}", str(file))   
    
def _process_probabilities(probabilities):
    """
    
    """
    prediction = np.argmax(probabilities)
    confidence = probabilities[prediction]
    return prediction, confidence
    
def _process_prediction(prediction, confidence, directory=None):
    """
    
    """    
    return {
        "species": _get_class(prediction, directory),
        "prediction": int(prediction),
        "confidence": confidence.item()
    }   


def model_fn(model_dir):
    """
    
    """
    print("Loading model ...")
    input_shape = 7
    
    model = PenguinModel(input_shape=input_shape)
    with open(Path(model_dir) / "model" / "001" / 'model.pth', 'rb') as f:
        model.load_state_dict(torch.load(f))
        
    for key, value in os.environ.items():
        print(f"{key}: {value}")
    
    return model
    
def input_fn(request_body, request_content_type, directory=None):
    """
    """    
    print(f"Processing input data...")
    
    if request_content_type in ("application/json", "application/octet-stream"):
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # JSON in that case.
        endpoint_input = json.loads(request_body)

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
    else:
        raise ValueError(f"Unsupported content type: {context.request_content_type or 'unknown'}")
        
    pipeline = _get_pipeline(directory)    
    df = pd.json_normalize(endpoint_input)
    result = pipeline.transform(df)
    result = pd.DataFrame(result)
    
    
    return result
    

def predict_fn(input_data, model):
    """
    """

    print(f"Sending input data to model to make a prediction...")
  
    with torch.no_grad():
        model.eval()
        out = model.predict(input_data)
    return out
    
def output_fn(predictions, directory=None, accept="application/json"):
    """
    """    
    print("Processing prediction received from the model...")
    
    results = list(map(_process_probabilities, predictions))
    print(results)
    predictions = list(map(lambda result: _process_prediction(result[0],result[1], directory), results))

    if accept == 'application/json':
        return json.dumps(predictions), accept
    raise Exception(f'Requested unsupported ContentType in Accept:{accept}')


Overwriting code/endpoint/inference.py


In [140]:
#  Test inference code
from preprocessor import preprocess
from endpoint.inference import *
import json 

samples = [
    {
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    },
    {
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    }
]

with tempfile.TemporaryDirectory() as directory:
    
    preprocess(
        base_directory=directory, 
        data_filepath=DATA_FILEPATH
    )

    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )
    
    request_content_type = "application/json"
    
    for sample in samples:
        model = model_fn(Path(directory))
        transformed_list = input_fn(json.dumps(sample), request_content_type, directory)
        output_np = predict_fn(transformed_list, model)
        predictions = output_fn(output_np,directory)
     



Epoch [1/50] - loss: 1.0838, val_accuracy: 0.4310
Epoch [2/50] - loss: 1.0833, val_accuracy: 0.4310
Epoch [3/50] - loss: 1.0827, val_accuracy: 0.4310
Epoch [4/50] - loss: 1.0822, val_accuracy: 0.4310
Epoch [5/50] - loss: 1.0817, val_accuracy: 0.4310
Epoch [6/50] - loss: 1.0812, val_accuracy: 0.4310
Epoch [7/50] - loss: 1.0807, val_accuracy: 0.4310
Epoch [8/50] - loss: 1.0803, val_accuracy: 0.4310
Epoch [9/50] - loss: 1.0798, val_accuracy: 0.4310
Epoch [10/50] - loss: 1.0793, val_accuracy: 0.4310
Epoch [11/50] - loss: 1.0788, val_accuracy: 0.4310
Epoch [12/50] - loss: 1.0783, val_accuracy: 0.4310
Epoch [13/50] - loss: 1.0778, val_accuracy: 0.4310
Epoch [14/50] - loss: 1.0773, val_accuracy: 0.4310
Epoch [15/50] - loss: 1.0768, val_accuracy: 0.4310
Epoch [16/50] - loss: 1.0764, val_accuracy: 0.4310
Epoch [17/50] - loss: 1.0759, val_accuracy: 0.4310
Epoch [18/50] - loss: 1.0754, val_accuracy: 0.4310
Epoch [19/50] - loss: 1.0749, val_accuracy: 0.4310
Epoch [20/50] - loss: 1.0744, val_accura

SageMaker's default TensorFlow inference container doesn't come with Scikit-Learn installed, so we need to provide a `requirements.txt` file with the libraries we want SageMaker to install in our endpoint.

In [141]:
%%writefile {ENDPOINT_CODE_FOLDER}/requirements.txt

numpy==1.19.5
pandas==1.1.5
scikit-learn==0.23.2
boto3

Overwriting code/endpoint/requirements.txt


In [142]:

# response = sagemaker_client.list_model_packages(
#     ModelPackageGroupName=MODEL_PACKAGE_GROUP,
#     ModelApprovalStatus="Approved",
#     SortBy="CreationTime",
#     MaxResults=1,
# )

# package = response["ModelPackageSummaryList"][0] if response["ModelPackageSummaryList"] else None
# package


# model_package = ModelPackage(
#     model_package_arn=package["ModelPackageArn"], 
#     sagemaker_session=sagemaker_session,
#     role=role, 
# )

repacked_model = PyTorchModel(
    name="penguins",
    model_data=(
        tune_model_step.get_top_model_s3_uri(top_k=0, s3_bucket=PipelineSession().default_bucket())
    ),
    framework_version="1.8",
    py_version="py36",
    sagemaker_session=PipelineSession(),
    role=role,
    entry_point="inference.py",
    source_dir=str(ENDPOINT_CODE_FOLDER),
    env={
        "PIPELINE_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["pipeline"].S3Output.S3Uri,
                "pipeline.pkl",
            ]
        ),
        "CLASSES_S3_LOCATION": Join(
            on="/",
            values=[
                preprocess_data_step.properties.ProcessingOutputConfig.Outputs["classes"].S3Output.S3Uri,
                "classes.csv",
            ]
        )
    }    
)


register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=repacked_model.register(
        model_package_group_name=model_package_group_name,
        approval_status="Approved",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.8",
    )
)

/opt/conda/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [143]:
%%writefile {CODE_FOLDER}/lambda.py

import os
import json
import boto3
import time

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_package_arn = event["model_package_arn"]
    endpoint_name = event["endpoint_name"]
    data_capture_percentage = event["data_capture_percentage"]
    data_capture_destination = event["data_capture_destination"]
    role = event["role"]
    
    timestamp = time.strftime("%m%d%H%M%S", time.localtime())
    model_name = f"penguins-model-{timestamp}"
    endpoint_config_name = f"penguins-endpoint-config-{timestamp}"

    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": model_package_arn
        }] 
    )

    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ],
        DataCaptureConfig={
            "EnableCapture": True,
            "InitialSamplingPercentage": data_capture_percentage,
            "DestinationS3Uri": data_capture_destination,
            "CaptureOptions": [
                {
                    'CaptureMode': "Input"
                },
                {
                    'CaptureMode': "Output"
                },
            ],
            "CaptureContentTypeHeader": {
                "JsonContentTypes": [
                    "application/json",
                    "application/octect-stream"
                ]
            }
        },
    )

    sagemaker.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully")
    }

Overwriting code/lambda.py


In [144]:
def create_lambda_role(role_name):
    try:
        response = iam_client.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description="Lambda Pipeline Role"
        )

        role_arn = response['Role']['Arn']

        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        iam_client.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )

        return role_arn

    except iam_client.exceptions.EntityAlreadyExistsException:
        response = iam_client.get_role(RoleName=role_name)
        return response['Role']['Arn']


lambda_role = create_lambda_role("lambda-pipeline-role")

In [145]:
data_capture_percentage = ParameterInteger(
    name="data_capture_percentage",
    default_value=100,
)

data_capture_destination = ParameterString(
    name="data_capture_destination",
    default_value=f"{S3_LOCATION}/monitoring/data-capture",
)

deploy_fn = Lambda(
    function_name="deploy_fn",
    execution_role_arn=lambda_role,
    script=str(CODE_FOLDER / "lambda.py"),
    handler="lambda.lambda_handler",
    timeout=600
)

deploy_fn.upsert()

# print(register_model_step.properties)
deploy_step = LambdaStep(
    name="deploy",
    lambda_func=deploy_fn,
    inputs={
        # We use the ARN of the model we registered to
        # deploy it to the endpoint.
        "model_package_arn": register_model_step.properties.ModelPackageArn,

        "endpoint_name": "penguins-endpoint",
        
        "data_capture_percentage": data_capture_percentage,
        "data_capture_destination": data_capture_destination,
        
        "role": role,
    }
)

In [146]:
# We'll send this payload to the endpoint. Notice how each line contains
# the information of a penguin. The endpoint will return the predictions
# for each of these lines.
# payload = """
# 0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0
# -0.7751048801481084, 0.8822689351285553,  -1.2168066120762704, 0.9226343641317372, 0.0, 1.0, 0.0
# -0.837387834894918, 0.3386660813829646, -0.26237731892812, -1.92351941317372, 0.0, 0.0, 1.0
# """

# We can now send the request to the endpoint and process the response.
# predictor = Predictor(endpoint_name=ENDPOINT)
# predictor.delete_endpoint()
# response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
# response = json.loads(response.decode("utf-8"))

# print(json.dumps(response, indent=2))
# print(f"\nSpecies: {np.argmax(response['predictions'], axis=1)}")

# Session 5 - Data Monitoring

In this session we'll set up a monitoring process to analyze the quality of the data our endpoint receives in production. For this, we will have SageMaker capture and evaluate the data observed by the endpoint.

To enable this functionality, we need a couple of steps:

1. Create a baseline to compare the real-time traffic.
2. Set up a schedule to continuously evaluate and compare against the baseline.

Notice that the Data Quality process uses the baseline dataset we generated during preprocessing. This baseline dataset is the same unprocessed train set in JSON format. We do this because we transformed the train data during the preprocessing step, but we need raw data because that's what the endpoint expects.

Check [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html) for a brief explanation of how to use SageMaker's Model Monitoring functionality. [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) is a much more extensive guide to monitoring in Amazon SageMaker.

Here is what the Pipeline will look like at the end of this session:

<img src='images/session5-pipeline.png' alt='Session 5 Pipeline' width="600">


In [147]:
import random

from datetime import datetime
from IPython.display import JSON

from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig, QualityCheckStep
from sagemaker.workflow.execution_variables import ExecutionVariables

from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.parameters import ParameterBoolean
from sagemaker.model import Model
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.s3 import S3Uploader


DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"

## Step 1 - Checking Captured Data

Let's check the S3 location where the endpoint stores the requests and responses that it receives.

Notice that it make take a few minutes for the first few files to show up in S3. Keep running the following line until you get some.

In [148]:
files = S3Downloader.list(data_capture_destination.default_value)[:3]
files

[]

These files contain the data captured by the endpoint in a SageMaker-specific JSON-line format. Each inference request is captured in a single line in the `jsonl` file. The line contains both the input and output merged together.

Let's read the first line from the first file:

In [149]:
if len(files):
    lines = S3Downloader.read_file(files[0])
    print(json.dumps(json.loads(lines.split("\n")[0]), indent=2))

## Step 2 - Generating a Baseline

Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the train set we generated in the preprocessing step.

We can configure the instance that will run the quality check using the [CheckJobConfig](https://sagemaker.readthedocs.io/en/v2.73.0/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.check_job_config.CheckJobConfig) class, and we can use the `DataQualityCheckConfig` class to configure the job.

In [150]:
data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=sagemaker_session,
        role=role,
    ),
    
    quality_check_config = DataQualityCheckConfig(
        # We will use the train dataset we generated during the preprocessing 
        # step to generate the data quality baseline.
        baseline_dataset=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,

        dataset_format=DatasetFormat.json(lines=True),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=model_package_group_name,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


# Session 6 - Model Monitoring

This session aims to set up a monitoring process to analyze the quality of the model predictions. For this, we need to generate ground truth for the data captured by the endpoint and compare it with a baseline performance.

Check [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html) for a brief explanation of how to use SageMaker's Model Monitoring functionality. [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) is a much more extensive guide to Model Monitoring in Amazon SageMaker.

Here is what the Pipeline will look like at the end of this session:

<img src='images/session6-pipeline.png' alt='Session 6 Pipeline' width="600">


In [151]:
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

from sagemaker.inputs import CreateModelInput, TransformInput
from sagemaker.transformer import Transformer
from sagemaker.workflow.steps import CreateModelStep, TransformStep

## Step 1 - Creating Test Predictions

To create a baseline to compare the model performance, we must create predictions for the test set and compare them with the predictions from the model. We can do this by running a Batch Transform Job to predict every sample from the test dataset. We can use a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) as part of the pipeline to run this job. You can check [Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) for more information about Batch Transform Jobs.

The Transform Step requires a model to generate predictions, so we need a Model Step that creates a model.

We also need to configure the [Batch Transform Job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) using a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform). This Batch Transform Job will run every sample from the training dataset through the model so we can compute the baseline metrics. We can use an instance of the [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html) class to configure the job.

In [152]:
create_model_step = ModelStep(
    name="create",
    display_name="create-model",
    step_args=repacked_model.create(
        instance_type="ml.m5.large"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    base_transform_job_name="transform",

    instance_type="ml.m5.large",
    instance_count=1,
    
    accept="application/json",
    strategy="SingleRecord",
    assemble_with="Line",
    
    output_path=f"{S3_LOCATION}/transform",
)

# Workaround for bug in SDK version 2.171.0
# https://github.com/aws/sagemaker-python-sdk/issues/3991
transformer._current_job_name = "transform"

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    transformer=transformer,
    inputs=TransformInput(
        
        # We will use the test dataset we generated during the preprocessing 
        # step to run it through the model and generate predictions.
        data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["test-baseline"].S3Output.S3Uri,

        join_source="Input",
        content_type="application/json",
        split_type="Line",
    ),
    cache_config=cache_config
)

## Step 2 - Generating a Baseline

Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the data we generated in the Transform Step.

In [153]:
model_quality_location = f"{S3_LOCATION}/monitoring/model-quality"

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=sagemaker_session,
        role=role,
    ),
    
    quality_check_config = ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,

        dataset_format=DatasetFormat.json(lines=True),

        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="MulticlassClassification",
        inference_attribute="$.SageMakerOutput.species",
        ground_truth_attribute="species",

        output_s3_uri=model_quality_location,
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=model_package_group_name,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


## Step 3 - Setting up Model Metrics

We can configure a new set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) using the results of the Data and Model Quality Steps.

In [154]:
model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

## Step 4 - Registering the Model

We need to redefine the Model Step to register the [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) so it takes into account the new metrics.

## Step 5 - Setting up the Condition Step

We only want to compute the model quality baseline if the model's performance is above the predefined threshold. The [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) will gate all necessary steps to compute the baseline. 

In [155]:
# We'll send this payload to the endpoint. Notice how each line contains
# the information of a penguin. The endpoint will return the predictions
# for each of these lines.
# payload = """
# 0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0
# -0.7751048801481084, 0.8822689351285553,  -1.2168066120762704, 0.9226343641317372, 0.0, 1.0, 0.0
# -0.837387834894918, 0.3386660813829646, -0.26237731892812, -1.92351941317372, 0.0, 0.0, 1.0
# """

# # We can now send the request to the endpoint and process the response.
# predictor = Predictor(endpoint_name=ENDPOINT)
# response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
# response = json.loads(response.decode("utf-8"))

# print(json.dumps(response, indent=2))
# print(f"\nSpecies: {np.argmax(response['predictions'], axis=1)}")

In [156]:
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[gte_approved],
    if_steps=[
        create_model_step, 
        generate_test_predictions_step,
        model_quality_baseline_step,
        register_model_step,
        deploy_step
    ],
    else_steps=[fail_step_min], 
)

## Step 6 - Setting up the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.

In [161]:
session6_pipeline = Pipeline(
    name="final-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        data_capture_percentage,
        data_capture_destination,
        accuracy_threshold,
        min_accuracy_threshold,
    ],
    steps=[
        preprocess_data_step,
        data_quality_baseline_step,
        tune_model_step,
        eval_model_step,
        condition_step,         
    ],
    pipeline_definition_config=pipeline_definition_config
)

session6_pipeline.upsert(role_arn=role)
try:
    response = boto3.client("sagemaker").delete_endpoint(
        EndpointName="penguins-endpoint",
    )
except:
    print("Endpoint does not exist")
    
# session6_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource
Using provided s3_resource
Endpoint does not exist


In [159]:

endpoint = 'penguins-endpoint'

# Read image into memory
payload=[{
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    },{
        "island": "Dream",
        "culmen_length_mm": 43.2,
        "culmen_depth_mm": 17.5,
        "flipper_length_mm": 175.0,
        "body_mass_g": 3500.0,
    },{
        "island": "Torgersen",
        "culmen_length_mm": 38.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 176.0,
        "body_mass_g": 3700.0,
    }]

predictor = Predictor(endpoint)
inference_response = predictor.predict(data=json.dumps(payload))
print (inference_response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/penguins-endpoint in account 253909639528 for more information.

In [ ]:
# Train my estimator
pytorch_estimator = PyTorch(entry_point=f'{CODE_FOLDER}/{ENDPOINT_CODE_FOLDER}/inference.py',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3')
pytorch_estimator.fit('s3://my_bucket/my_training_data/')

# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = pytorch_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

# `data` is a NumPy array or a Python list.
# `response` is a NumPy array.
response = predictor.predict(data)